# Airline Data Modeling
___

This will be a classification model intended to predict budget vs. non-budget commercial passenter airlines based on safety data collected from the NSTB aviation accident [database](https://www.ntsb.gov/Pages/AviationQueryV2.aspx). The bud get airlines will be the target and assigned as '1'.

Ultimately, we were unsuccessful in try to predict a budget vs. non-budget airline based on data from the NTSB aviation accident database using logistic regression and CountVectorizer.

## Contents
---
- [Data Cleaning & Setting Target](#Data-Cleaning-&-Setting-Target)
- [Train/Test/Split & Base Model](#Train/Test/Split-&-Base-Model)
- [Feature Engineering](#Feature-Engineering)
- [Modeling](#Modeling)
- [Predictions and Interpretation](#Predictions-and-Interpretation)
- [Simplier Logistic Regression Model](#Simplier-Logistic-Regression-Model)
- [Just For Fun - Aircraft Make & Airline](#Just-For-Fun---Aircraft-Make-&-Airline)

### Data Prepping & Setting Target
___

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore", message="Found unknown categories in columns")

#### Budget Airlines (Target)

* Alaska
* Frontier
* JetBlue
* Allegiant
* Spirit
* Sun Country
* Southwest

#### Non-Budget Airlines

* American
* United
* Delta
* US Airways 
* Continental
* Hawaiian

Note: US Airways merged with American and Continental merged with United.

Read in and drop columns we won't need for modeling.  Kept tail number and lat/long for research purposes only.  Tail number is great for looking up plane models.


In [2]:
airlines_df = pd.read_csv('./data/text_processed_aviation_data.csv')

In [3]:
airlines_df.columns

Index(['event_type', 'event_date', 'tail_number', 'highest_injury_level',
       'fatal_injury_count', 'serious_injury_count', 'minor_injury_count',
       'probable_cause', 'latitude', 'longitude', 'airport_id', 'operator',
       'make', 'aircraft_damage', 'model'],
      dtype='object')

In [4]:
columns_to_drop = ['tail_number', 'event_date', 'latitude', 'longitude']

In [5]:
airlines_df.drop(columns = columns_to_drop, inplace = True)

In [6]:
airlines_df['operator'].value_counts()

operator
american       129
delta          102
united          89
southwest       62
continental     54
us airways      52
alaska          22
frontier        15
jetblue         11
hawaiian         7
allegiant        6
spirit           5
sun country      4
Name: count, dtype: int64

Binarize the budget and non-budget airlines.

In [7]:
targets = {'american':0,
              'delta':0,
              'united':0,
              'southwest':1,
              'continental':0,
              'us airways':0,
              'alaska':1,
              'frontier':1,
              'jetblue':1,
              'hawaiian':0,
              'allegiant':1,
              'spirit':1,
              'sun country':1
             }

airlines_df['operator'] = airlines_df['operator'].replace(targets)

In [8]:
# Note that the values should be stratified for modeling as they are uneven.
airlines_df['operator'].value_counts()

operator
0    433
1    125
Name: count, dtype: int64

### Train/Test/Split & Base Model
___

In [9]:
# Set X and y
X = airlines_df.drop(columns = 'operator')
y = airlines_df['operator']

In [10]:
# Train/Test/Split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state=24,
                                                   stratify=y)  

**What is the base model?**
* There is a **77%** chance of correctly predicting a non-budget airline.

In [11]:
y_train.value_counts(normalize = True)

operator
0    0.77512
1    0.22488
Name: proportion, dtype: float64

In [12]:
#Ensuring that stratify is maintaining the skew between the types of airlines.
y_train.value_counts(normalize = True), y_test.value_counts(normalize = True)

(operator
 0    0.77512
 1    0.22488
 Name: proportion, dtype: float64,
 operator
 0    0.778571
 1    0.221429
 Name: proportion, dtype: float64)

### Feature Engineering
___

All of the columns in the dataset are categorical except for the fatal/serious/minor injury counts.  Therefore, they shall be one-hot encoded. We will use a standard scaler on the injury counts only.  I am not sure if the highest_injury_count column is helpful when pair with quantitative injury counts. Possibly something to explore given time.

In [13]:
#Set up a column transformer

ohe_columns = ['event_type', 'highest_injury_level', 'airport_id', 'make', 'aircraft_damage', 'model']
ss_columns = ['fatal_injury_count','serious_injury_count', 'minor_injury_count']


ctx = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), ohe_columns),
    ('sc', StandardScaler(), ss_columns),
    ('cvec', CountVectorizer(), 'probable_cause')
], remainder='passthrough', verbose_feature_names_out=False)


As always, a wide variety of parameters were tested.  Liblinear was chosen due to convergence errors. Below are the best settings.

In [14]:
pipe_params = {
    'ctx__cvec__max_features': [200],
    'ctx__cvec__min_df': [3],
    'ctx__cvec__max_df': [0.50],
    'ctx__cvec__ngram_range': [(1,3)],
    'lr__C': [10],
    'lr__penalty': ['l2'],
    'lr__solver': ['liblinear']
}

In [15]:
pipe = Pipeline([
    ('ctx', ctx),
    ('lr', LogisticRegression(random_state = 24))
])


In [16]:
gs = GridSearchCV(pipe,
                  pipe_params,
                  cv = 5)

## Modeling
___

In [17]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ctx',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['event_type',
                                                                          'highest_injury_level',
                                                                          'airport_id',
                                                                          'make',
                                                                          'aircraft_damage',
                                                                          'model']),
                                                                        ('sc',
                                                                         StandardScaler(),
                                                                         ['fatal_injury_count',
                                                                          'serious_injury_count',
                                                                          'minor_injury_count']),
                                                                        ('cvec',
                                                                         CountVectorizer(),
                                                                         'probable_cause')],
                                                          verbose_feature_names_out=False)),
                                       ('lr',
                                        LogisticRegression(random_state=24))]),
             param_grid={'ctx__cvec__max_df': [0.5],
                         'ctx__cvec__max_features': [200],
                         'ctx__cvec__min_df': [3],
                         'ctx__cvec__ngram_range': [(1, 3)], 'lr__C': [10],
                         'lr__penalty': ['l2'], 'lr__solver': ['liblinear']})

In [18]:
gs.score(X_train, y_train)

0.9952153110047847

In [19]:
gs.score(X_test, y_test)

0.8142857142857143

73% is less than the baseline score of 78% and the model is extremely overfit.

In [20]:
#What was the best score?
print(f'The best accuracy score in all models tested in grid search is {round(gs.best_score_* 100,2)}%')

The best accuracy score in all models tested in grid search is 78.01%


In [21]:
# Breakdown of the parameters chosen to make the best model.
gs.best_params_

{'ctx__cvec__max_df': 0.5,
 'ctx__cvec__max_features': 200,
 'ctx__cvec__min_df': 3,
 'ctx__cvec__ngram_range': (1, 3),
 'lr__C': 10,
 'lr__penalty': 'l2',
 'lr__solver': 'liblinear'}

## Predictions and Interpretation
___

In [22]:
#Training set results
print(f'Training accuracy score is at {round(gs.score(X_train, y_train)* 100,2)}%')

Training accuracy score is at 99.52%


In [23]:
# Testing set results is at 72% - Null model is at 77%, so this model
# is unable to predict airline based on safety data.
print(f'Testing accuracy score is at {round(gs.score(X_test, y_test)* 100,2)}%')

Testing accuracy score is at 81.43%


In [24]:
# Calculating model predictions (budget airlines = 1 and non-budget = 0)
# and the probability of each raw value.
pred = gs.predict(X_test)
pred[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
      dtype=int64)

In [25]:
gs.predict_proba(X_test)[:20]

array([[9.99999682e-01, 3.17780129e-07],
       [9.82589895e-01, 1.74101051e-02],
       [9.91873547e-01, 8.12645303e-03],
       [9.99922294e-01, 7.77064036e-05],
       [9.50208272e-01, 4.97917281e-02],
       [9.84747837e-01, 1.52521627e-02],
       [9.64073954e-01, 3.59260463e-02],
       [9.34422895e-01, 6.55771051e-02],
       [9.93846911e-01, 6.15308913e-03],
       [8.80572534e-01, 1.19427466e-01],
       [7.24509805e-01, 2.75490195e-01],
       [2.17480714e-01, 7.82519286e-01],
       [9.95328140e-01, 4.67186032e-03],
       [1.20819649e-01, 8.79180351e-01],
       [8.21154985e-01, 1.78845015e-01],
       [4.37384606e-02, 9.56261539e-01],
       [9.98154821e-01, 1.84517856e-03],
       [9.64154117e-01, 3.58458830e-02],
       [9.87357848e-01, 1.26421516e-02],
       [9.81008857e-01, 1.89911432e-02]])

In [26]:
# Accuracy Score - This is worse than the baseline model.
print(f'This model has an accuracy score of {round(metrics.accuracy_score(y_test, pred) * 100,2)}%')

This model has an accuracy score of 81.43%


### PCA
___

The testing score is the same as the baseline.  Seeing a trend of not being able to create a model that does better than the baseline with respect to predicting budget vs. non-budget airlines using accident data.

In [27]:
#Set up a column transformer#Set up a column transformer

ohe_columns = ['event_type', 'highest_injury_level', 'airport_id', 'make', 'aircraft_damage', 'model']
ss_columns = ['fatal_injury_count','serious_injury_count', 'minor_injury_count']


ctx = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), ohe_columns),
    ('sc', StandardScaler(), ss_columns),
    ('cvec', CountVectorizer(), 'probable_cause')
], remainder='passthrough', verbose_feature_names_out=False)


In [28]:
pipe_params = {
    'ctx__cvec__max_features': [200],
    'ctx__cvec__min_df': [3],
    'ctx__cvec__max_df': [0.50],
    'ctx__cvec__ngram_range': [(1,3)],
    'lr__C': [10],
    'lr__penalty': ['l2'],
    'lr__solver': ['liblinear'],
    'pca__n_components': range(4,41,2)
}

In [29]:
pipe = Pipeline([
    ('ctx', ctx),
    ('pca', PCA()),
    ('lr', LogisticRegression(random_state = 24))
])


In [30]:
gs = GridSearchCV(pipe,
                  param_grid=pipe_params,
                  n_jobs = -1)

In [31]:
gs.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('ctx',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['event_type',
                                                                          'highest_injury_level',
                                                                          'airport_id',
                                                                          'make',
                                                                          'aircraft_damage',
                                                                          'model']),
                                                                        ('sc',
                                                                         StandardScaler(),
                                                                         ['fatal_injury_count',
                                                                          'serious_injury_count',
                                                                          'minor_injury_co...
                                                                         CountVectorizer(),
                                                                         'probable_cause')],
                                                          verbose_feature_names_out=False)),
                                       ('pca', PCA()),
                                       ('lr',
                                        LogisticRegression(random_state=24))]),
             n_jobs=-1,
             param_grid={'ctx__cvec__max_df': [0.5],
                         'ctx__cvec__max_features': [200],
                         'ctx__cvec__min_df': [3],
                         'ctx__cvec__ngram_range': [(1, 3)], 'lr__C': [10],
                         'lr__penalty': ['l2'], 'lr__solver': ['liblinear'],
                         'pca__n_components': range(4, 41, 2)})

In [32]:
gs.best_params_

{'ctx__cvec__max_df': 0.5,
 'ctx__cvec__max_features': 200,
 'ctx__cvec__min_df': 3,
 'ctx__cvec__ngram_range': (1, 3),
 'lr__C': 10,
 'lr__penalty': 'l2',
 'lr__solver': 'liblinear',
 'pca__n_components': 10}

In [33]:
#What was the best score?
print(f'The best accuracy score in all models tested in grid search is {round(gs.best_score_* 100,2)}%')

The best accuracy score in all models tested in grid search is 77.76%


In [34]:
#Training set results
print(f'Training accuracy score is at {round(gs.score(X_train, y_train)* 100,2)}%')

Training accuracy score is at 77.75%


In [35]:
# Testing set results is at 74% - Null model is at 78%, so this model
# is unable to predict airline based on safety data.
print(f'Testing accuracy score is at {round(gs.score(X_test, y_test)* 100,2)}%')

Testing accuracy score is at 78.57%


### Simplier Logistic Regression Model
___

Clearly the previous model is overfit and so we'll try to make a model with less features in an attempt to simplify it. Will consider only using highest_injury_level and aircraft_damage.

In [36]:
airlines_df.columns

Index(['event_type', 'highest_injury_level', 'fatal_injury_count',
       'serious_injury_count', 'minor_injury_count', 'probable_cause',
       'airport_id', 'operator', 'make', 'aircraft_damage', 'model'],
      dtype='object')

In [37]:
X = airlines_df[['highest_injury_level','aircraft_damage']]
y = airlines_df['operator']

In [38]:
X
y

0      0
1      0
2      1
3      1
4      1
      ..
553    0
554    0
555    0
556    0
557    0
Name: operator, Length: 558, dtype: int64

In [39]:
# Train/Test/Split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state=24,
                                                   stratify=y)  

In [40]:
y_train.value_counts(normalize = True)

operator
0    0.77512
1    0.22488
Name: proportion, dtype: float64

In [41]:
#Ensuring that stratify is maintaining the slight skew between the types of airlines.
y_train.value_counts(normalize = True), y_test.value_counts(normalize = True)

(operator
 0    0.77512
 1    0.22488
 Name: proportion, dtype: float64,
 operator
 0    0.778571
 1    0.221429
 Name: proportion, dtype: float64)

In [42]:
simple_ohe_columns = ['highest_injury_level', 'aircraft_damage']

ctx = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), simple_ohe_columns),
], remainder='passthrough',
    verbose_feature_names_out = False
)

In [43]:
pipe_params = {
    'lr__C': [0.05],
    'lr__penalty': ['l2'],
    'lr__solver': ['liblinear']
}

In [44]:
pipe = Pipeline([
    ('ctx', ctx),
    ('lr', LogisticRegression())
])


In [45]:
gs = GridSearchCV(pipe,
                  pipe_params,
                  cv = 5)

In [46]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ctx',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['highest_injury_level',
                                                                          'aircraft_damage'])],
                                                          verbose_feature_names_out=False)),
                                       ('lr', LogisticRegression())]),
             param_grid={'lr__C': [0.05], 'lr__penalty': ['l2'],
                         'lr__solver': ['liblinear']})

In [47]:
gs.score(X_test, y_test)

0.7785714285714286

In [48]:
# This is better than the first logistic regression model, but no better than the baseline.
print(f'The best accuracy score in all models tested in grid search is {round(gs.best_score_* 100,2)}%')

The best accuracy score in all models tested in grid search is 77.51%


In [49]:
# Breakdown of the parameters chosen to make the best model.
gs.best_params_

{'lr__C': 0.05, 'lr__penalty': 'l2', 'lr__solver': 'liblinear'}

In [50]:
# Calculating model predictions (budget airlines = 1 and non-budget = 0)
# and the probability of each raw value.
pred = gs.predict(X_test)
pred[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [51]:
gs.predict_proba(X_test)[:20]

array([[0.75554328, 0.24445672],
       [0.75554328, 0.24445672],
       [0.74351255, 0.25648745],
       [0.74351255, 0.25648745],
       [0.74351255, 0.25648745],
       [0.70544418, 0.29455582],
       [0.75895665, 0.24104335],
       [0.68798101, 0.31201899],
       [0.75895665, 0.24104335],
       [0.75554328, 0.24445672],
       [0.72395714, 0.27604286],
       [0.72395714, 0.27604286],
       [0.73289417, 0.26710583],
       [0.75554328, 0.24445672],
       [0.75895665, 0.24104335],
       [0.74351255, 0.25648745],
       [0.73289417, 0.26710583],
       [0.74351255, 0.25648745],
       [0.70713588, 0.29286412],
       [0.75554328, 0.24445672]])

In [52]:
# Accuracy Score - This is about the same as the baseline model.
print(f'This model has an accuracy score of {round(metrics.accuracy_score(y_test, pred) * 100,2)}%')

This model has an accuracy score of 77.86%


 Even when simplified, the model score doesn't do better than the baseline.  Other notebooks will explore bagging, random forests, etc.

### Just For Fun - Aircraft Make & Airline
___

Clearly the data we've collected doesn't seem to offer any predictive power with respect to our target.  We decided to try a very simple model of predicting a budget vs. non-budget aircraft using the aircraft make feature.  Both the baseline and test score were about 78% - the same as the baseline.

Spoilers:  The test score is the same as the train score - which is the same as the baseline score.

In [53]:
X = airlines_df[['make']]
y = airlines_df['operator']

In [54]:
X

,make
0,boeing
1,boeing
2,boeing
3,boeing
4,airbus
...,...
553,mcdonnell_douglas
554,boeing
555,boeing
556,boeing


In [55]:
# Train/Test/Split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state=24,
                                                   stratify=y)  

In [56]:
ctx = ColumnTransformer(
    transformers= [
        ('ohe', OneHotEncoder(drop='first',
                             handle_unknown='ignore',
                             sparse_output=False),
        ['make']),
    ], verbose_feature_names_out=True,
    remainder='passthrough'
)

In [57]:
pipe_params = {
    'lr__C': [0.05],
    'lr__penalty': ['l2'],
    'lr__solver': ['liblinear']
}

In [58]:
pipe = Pipeline([
    ('ctx', ctx),
    ('lr', LogisticRegression())
])


In [59]:
gs = GridSearchCV(pipe,
                  pipe_params,
                  cv = 5)

In [60]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ctx',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['make'])])),
                                       ('lr', LogisticRegression())]),
             param_grid={'lr__C': [0.05], 'lr__penalty': ['l2'],
                         'lr__solver': ['liblinear']})

In [61]:
gs.score(X_train, y_train)

0.7751196172248804

In [62]:
gs.score(X_test, y_test)

0.7785714285714286